In [ ]:
%pip install transformers torch pillow


In [ ]:
from transformers import ViTImageProcessor, ViTForImageClassification
from PIL import Image
import torch

image_path = "/content/cat.jpg"
image = Image.open(image_path).convert("RGB")

processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224")

inputs = processor(images=image, return_tensors="pt")
print(inputs)

with torch.no_grad():
    outputs = model(**inputs)

logits = outputs.logits
idx = logits.argmax(-1).item()
label = model.config.id2label[idx]

print("Predicted class index:", idx)
print("Predicted label:", label)


FileNotFoundError: [Errno 2] No such file or directory: '/content/cat.jpg'

In [ ]:
import torch
import timm
from PIL import Image
import torchvision.transforms as T

# ---------------------------------------
# 1. Load your local image
# ---------------------------------------
image_path = "/content/cat.jpg"   # <-- CHANGE THIS
image = Image.open(image_path).convert("RGB")

# ---------------------------------------
# 2. Define transforms (timm expects this style)
# ---------------------------------------
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225)
    )
])

img_tensor = transform(image).unsqueeze(0)   # Add batch dimension

# ---------------------------------------
# 3. Load a pre-trained ViT from TIMM
# ---------------------------------------
model = timm.create_model("vit_base_patch16_224", pretrained=True)
model.eval()   # inference mode

# ---------------------------------------
# 4. Do forward pass
# ---------------------------------------
with torch.no_grad():
    outputs = model(img_tensor)

# ---------------------------------------
# 5. Read class index
# ---------------------------------------
idx = outputs.argmax(-1).item()

# ---------------------------------------
# 6. Get full human-readable class label
# ---------------------------------------
labels = timm.data.resolve_data_config({}, model=model)["label"]
# but timm doesn't include labels for ImageNet by default, so we load manually:
import json, urllib.request

url = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet_simple_labels.json"
imagenet_labels = json.load(urllib.request.urlopen(url))

print("Predicted class index:", idx)
print("Predicted label:", imagenet_labels[idx])


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

KeyError: 'label'